# YANS 2025 Hackathon SFT

YANS 2025 で開催されるハッカソンの SFT サンプルコードです。  
ベースラインとなるデータセットの読み込み、学習、評価を行います。

## 環境設定

In [1]:
# 学習後のモデル推論用に vLLM をインストール
!pip install vllm
# ハッカソン用のコードをインストール
!pip install git+https://github.com/YANS-official/yans-2025-hackathon.git
# OpenAI API をノートブック上で使用する場合はライブラリをインストール
!pip install openai

  Cloning https://github.com/YANS-official/yans-2025-hackathon.git to /tmp/pip-req-build-rw1vsnvf
  Running command git clone --filter=blob:none --quiet https://github.com/YANS-official/yans-2025-hackathon.git /tmp/pip-req-build-rw1vsnvf
  Resolved https://github.com/YANS-official/yans-2025-hackathon.git to commit ceb16b01e67294ea8bf113c41bf566cfe0d87d40
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## データセットの読み込み

ベースラインとなる学習データセット及び開発セットは JSON ファイル形式で提供しています。

以下の YANS 提供の GitHub リポジトリの `datasets` フォルダに格納されています。

https://github.com/YANS-official/yans-2025-hackathon

### 学習セット

500件です。

本ハッカソンの主眼はデータ作成です。
本データセットから派生させて学習データを作成しても構いませんし、本データセットを使用せずに新たにデータを作成しても構いません。

### 検証セット

400件です。
開発中のモデルの性能を測定することに使います。

開発中は、本ファイルに対する推論結果をリーダーボードに提出してください。

なお、モデルの最終評価は別のテストセットで行います。
基本的な問題の分布は検証セットと同様です。

In [2]:
# YANS 提供のリポジトリをクローン
!git clone https://github.com/YANS-official/yans-2025-hackathon

Cloning into 'yans-2025-hackathon'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 81 (delta 33), reused 67 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 199.54 KiB | 6.05 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [3]:
# 先頭五件を表示
!head -n 5 yans-2025-hackathon/datasets/hackathon-2025-math-train.jsonl

{"id": "train-001", "question": "魚を毎日2匹ずつ買っていたら、1週間で何匹になるでしょうか。", "answer_number": 14}
{"id": "train-002", "question": "魚を毎日2匹ずつ購入しています。これを7日間継続した場合の累積数は？", "answer_number": 14}
{"id": "train-003", "question": "たかしさんは自分の家族のためにスパゲッティとミートボールの食事を準備しています。たかしさんのミートボールのレシピでは、1個のミートボールにつき1/8ポンドのひき肉が必要です。たかしさんの家族は彼を含めて8人います。彼が4ポンドのひき肉を使ってミートボールを作り、家族のそれぞれが等しい数のミートボールを食べる場合、たかしさんは何個のミートボールを食べますか？", "answer_number": 4}
{"id": "train-004", "question": "1冊12ページの本を3冊読みました。全部で何ページ読んだことになりますか？", "answer_number": 36}
{"id": "train-005", "question": "慶子さんは、9匹の猫に1匹ずつえさをやりました。そのときのえさの個数はいくつでしょうか。", "answer_number": 9}


## SFT 学習

運営の提供する `train_sft` 関数を用いて学習を行ってください。

### レギュレーション
- 学習データのサンプル数は最大 500 件まで
- 学習データ内の文字数は最大50万文字まで
- 学習に用いるモデルは [SakanaAI/TinySwallow-1.5B](https://huggingface.co/SakanaAI/TinySwallow-1.5B) に限る

バッチサイズや学習率といった学習時のハイパーパラメータは事前の実験で適切なものにセットしていますが、関数から指定できるものに限り、自由に変えていただいて構いません。

In [4]:
from yans_2025_hackathon import train_sft

# 学習用の関数の説明を表示
help(train_sft)

Help on function train_sft in module yans_2025_hackathon.train_sft:

train_sft(train_dataset: list[dict[str, str]], save_dir: str, model: str = 'SakanaAI/TinySwallow-1.5B', batch_size: int = 4, local_batch_size: int = 1, learning_rate: float = 2e-06, num_train_epochs: int = 3, use_lora: bool = False, training_arguments: dict | None = None, trainer_kwargs: dict | None = None)
    YANS 2025 ハッカソンで使用する SFT（Supervised Fine-tuning）を実行する関数。
    学習データにはサイズ制限があり、最大500サンプル、総文字数50万文字まで。

    Args:
        train_dataset (list[dict[str, str]]): 学習用データセット。以下の形式を持つ辞書のリスト。
            [
                {"question": "質問文", "answer": "回答文"},
                ...
            ]
        save_dir (str): 学習済みモデルの保存先ディレクトリ。
        model (str, optional): 使用する事前学習モデル。デフォルトは"SakanaAI/TinySwallow-1.5B"。
        batch_size (int, optional): バッチサイズ。デフォルトは4。
        local_batch_size (int, optional): デバイスごとのローカルバッチサイズ。デフォルトは1。
        learning_rate (float, optional): 学習率。デフォルトは2e-6。
        num_train_epochs (int, opt

学習時のサンプルは以下の形式にフォーマットされている必要があります。

```python
[
  {"question": "1+1は？", "answer": "2"},
  ...
]
```

ここでは例として、ベースラインとなるデータセットを上記の形に加工します。

In [5]:
import json

# JSONL ファイルからデータを読み込み
train_dataset: list[dict[str, str]] = []
with open("yans-2025-hackathon/datasets/hackathon-2025-math-train.jsonl") as f:
    for line in f:
        item = json.loads(line)
        # answer_number: int を str に変換する
        train_dataset.append(
            {"question": item["question"], "answer": str(item["answer_number"])}
        )
print(train_dataset[0])

{'question': '魚を毎日2匹ずつ買っていたら、1週間で何匹になるでしょうか。', 'answer': '14'}


In [6]:
from yans_2025_hackathon import train_sft

train_sft(
    train_dataset=train_dataset,
    save_dir="results",  # 学習後のモデルの保存場所
    # 無料版 GPU を使用する場合は以下のオプションをつける
    # use_lora = True,  # LoRA を使用して低メモリで学習
    # learning_rate = 2e-5,  # LoRA の学習率は高めが良い
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.816100
20,1.487100
30,1.091500
40,1.000200
50,1.011800
60,0.986100
70,1.078100
80,0.887300
90,0.904200
100,0.971800


Saving model to results


`save_dir` で指定したディレクトリにモデルが保存されています。

In [7]:
# 保存フォルダの中身を確認
!ls results

added_tokens.json	model-00001-of-00002.safetensors  tokenizer.json
chat_template.jinja	model-00002-of-00002.safetensors  training_args.bin
config.json		model.safetensors.index.json	  vocab.json
generation_config.json	special_tokens_map.json
merges.txt		tokenizer_config.json


## モデルの評価

学習したモデルのパフォーマンスを開発セットで測定しましょう。
高速な推論のために [vLLM](https://github.com/vllm-project/vllm) ライブラリを使用します。

訓練直後に、ディスクに保存されたモデルを vLLM で読み込むとメモリが足りずにエラーがでます。  
ノートブック上部の「ランタイム > ランタイムを再起動する」を実行してメモリを解放してください。

In [1]:
from vllm import LLM

model = LLM(
    model="./results",  # 学習したモデルのパスを指定
    max_model_len=2048,  # モデルが処理する最大系列長
    max_num_seqs=16,  # モデルが一度に処理するバッチ数（大きすぎると落ちる場合あり）
)

INFO 09-15 01:35:37 [__init__.py:216] Automatically detected platform cuda.
INFO 09-15 01:35:38 [utils.py:328] non-default args: {'max_model_len': 2048, 'max_num_seqs': 16, 'disable_log_stats': True, 'model': './results'}
INFO 09-15 01:35:56 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 09-15 01:35:56 [__init__.py:2764] Downcasting torch.float32 to torch.bfloat16.
INFO 09-15 01:35:56 [__init__.py:1815] Using max model len 2048
INFO 09-15 01:35:59 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 09-15 01:35:59 [__init__.py:2974] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 09-15 01:36:56 [llm.py:295] Supported_tasks: ['generate']
INFO 09-15 01:36:56 [__init__.py:36] No IOProcessor plugins requested by the model


In [2]:
# 簡単な問題で動作確認
batch_responses = model.chat([{"role": "user", "content": "1 + 1 はなんですか？"}])
print(batch_responses[0].outputs[0].text)

INFO 09-15 01:36:56 [chat_utils.py:538] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
WARNING 09-15 01:36:56 [__init__.py:1695] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

2


In [3]:
import json

# JSONL ファイルからデータを読み込み
dev_dataset: list[dict[str, str]] = []
with open("yans-2025-hackathon/datasets/hackathon-2025-math-dev.jsonl") as f:
    for line in f:
        item = json.loads(line)
        dev_dataset.append(item)
print(dev_dataset[0])

{'id': 'dev-001', 'question': '山本さんはキャンプ場の管理者で、キャンプ場に設置されたテントの数を確認する任務を負っています。ある日、彼はキャンプ場の北側に100張のテントを数え、その2倍のテントをキャンプ場の東側に数えました。キャンプ場の中心部には、北側のテントの数の4倍のテントがありました。また、南側には200張のテントを数えました。キャンプ場には合計で何張のテントがありますか？', 'answer_number': 900}


In [4]:
from vllm import SamplingParams

# 入力を chat 形式に変換する
dev_inputs = [[{"role": "user", "content": item["question"]}] for item in dev_dataset]
# モデルから出力を得る
dev_batch_responses = model.chat(
    dev_inputs,
    SamplingParams(
        temperature=0.0,  # 貪欲デコーディング
        max_tokens=1024,  # 最大生成トークン数
    ),
)

Adding requests:   0%|          | 0/400 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/400 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [6]:
from yans_2025_hackathon import evaluate_response

# 正解率を測定
num_total = 0
num_correct = 0
for resopnse, item in zip(dev_batch_responses, dev_dataset):
    response_text = resopnse.outputs[0].text
    answer_number = item["answer_number"]
    is_correct = evaluate_response(response_text, answer_number)

    num_total += 1
    num_correct += int(is_correct)
print("正解率", num_correct / num_total)

正解率 0.095


## リーダーボードへの提出

リーダーボードには、以下のようにデータ ID（`"id"`）と、モデルの回答（`"answer"`）を含む JSONL ファイルを提出してください。

```jsonl
{"id": "...", "answer": "..."}
{"id": "...", "answer": "..."}
```

`id` の値を用いて、適切な答えとの照合を行いますので、順番は気にしなくても構いません。

In [8]:
import json

# モデルの出力と、開発データセットのIDをまとめ、JSONL ファイルとして書き出す
with open("predictions.jsonl", "w") as f:
    for response, item in zip(dev_batch_responses, dev_dataset):
        f.write(
            json.dumps(
                {"id": item["id"], "answer": response.outputs[0].text},
                ensure_ascii=False,
            )
            + "\n"
        )

作成したファイルは、ノートブック左の 📁 アイコンをクリックし、そこに表示されるファイル一覧からダウンロードすることができます（[参考](https://www.kikagaku.co.jp/kikagaku-blog/google-colab-file/#i-3)）。

## OpenAI API の使い方

データを加工したり、作成するために OpenAI API を使うことができます。  
ChatGPT からの出力を得る方法として、以下のコードを参考にしてください。

In [7]:
from openai import OpenAI

client = OpenAI(
    api_key="sk-...",  # 配布された API キーを使用する
)

response = client.responses.create(
    model="gpt-4o-mini",  # 使用可能モデルは gpt-4o-mini のみ
    instructions="数学の問題を解いてください。応答の末尾に必ず「答え:」という形式で解答となる数字を出力してください。",
    input="たけしさんは飴を４個もらいましたが、２個食べました。残りの飴の数は何個？",
)

print(response.output_text)

たけしさんは最初に4個の飴をもらい、2個を食べました。

残りの飴の数は、4個 - 2個 = 2個です。

答え: 2
